In [442]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


dataset = pd.read_table('Data/data_banknote_authentication.txt', sep = ",", header=None)
X = dataset.iloc[:, :-1].values
t = dataset.iloc[:, -1].values

# split the training set, validation set and test set of ratio 6:2:2
X_train, X_test, t_train, t_test = train_test_split(X, t, test_size=0.2, random_state=8775)
X_train, X_valid, t_train, t_valid = train_test_split(X_train, t_train, test_size=0.25, random_state=8775)

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)
X_valid = sc.transform(X_valid)


In [443]:
def ReLU(x):
    return x * (x > 0)

def d_ReLU(x):
    return 1. * (x > 0)

def standard_normal(matrix):    # mean=0, stdev=1
    return np.random.standard_normal(size=matrix.shape)

def random_integers(matrix):    # random 1 -1
    opt = np.random.choice([0,1],size=matrix.shape)
    return np.where(opt==0, -1, opt)

def pattern(matrix):    # pattern matrix of 1 -1 1 -1 1 -1
    for m in range (matrix.shape[0]):

        if np.matrix.ndim != 1:
            for n in range (matrix.shape[1]):
                matrix[m][n] = (m+n)%2
        else:
            matrix[m] = m%2

    return np.where(matrix==0, -1, matrix)

def cross_entropy(y,t):
    eps = np.finfo(float).eps
    cross_entropy = -np.sum(t * np.log(y + eps))/len(t)
    return cross_entropy


def NNClassifier(X, t, hidden_layer_sizes, initilize, epochs, learning_rate):
    layer_1_size = hidden_layer_sizes[0]
    layer_2_size = hidden_layer_sizes[1]
    
    output = np.zeros(len(X))
    output_best = np.zeros(len(X))

    # initilize the matrix for best weights
    w_1_best = np.ones((layer_1_size, 5))
    w_2_best = np.ones((layer_2_size, layer_1_size+1))
    w_3_best = np.ones((1, layer_2_size+1))

    # initilize weights for training
    w_1 = initilize(w_1_best)
    w_2 = initilize(w_2_best)
    w_3 = initilize(w_3_best)

    j=0
    loss = np.ones(epochs)*np.inf

    while(j<epochs):

        X,t = shuffle(X,t)      # shuffle the set

        for i in range (len(X)):    # loop through all examples 
            input = X[i]

            # forward pass
            z_1 = np.dot(w_1, np.insert(input,0,1).T)
            h_1 = ReLU(z_1)

            z_2 = np.dot(w_2, np.insert(h_1,0,1).T)
            h_2 = ReLU(z_2)

            z_3 = np.dot(w_3, np.insert(h_2,0,1).T)

            output[i] = np.power((1 + np.exp(-z_3)), -1)
            #output[i] = np.rint( np.power((1 + np.exp(-z_3)), -1) )

            # backward pass
            dz_3 = -output[i]+ np.power((1 + np.exp(-z_3)), -1)
            gw_3 = dz_3*np.insert(h_2.T,0,1)
            gz_2 = np.multiply(d_ReLU(z_2),np.dot( np.delete(w_3,0,1).T, dz_3))

            gw_2 = gz_2.reshape(layer_2_size,1)*np.insert(h_1.T,0,1)
            gz_1 = np.multiply(d_ReLU(z_1),np.dot( np.delete(w_2,0,1).T, gz_2))

            gw_1 = gz_1.reshape(layer_1_size,1)*np.insert(input.T,0,1)

            # update w
            w_3 = np.subtract(w_3, np.dot(learning_rate,gw_3))
            w_2 = np.subtract(w_2, np.dot(learning_rate,gw_2))
            w_1 = np.subtract(w_1, np.dot(learning_rate,gw_1))

        # calcuate loss of these particular group of weights
        loss[j] = cross_entropy(output,t)   

        if np.argmin(loss)==j:  # if the current result is minimum, update best weights
            w_1_best = w_1
            w_2_best = w_2
            w_3_best = w_3
            output_best = output


        j=j+1

    

    return output_best, np.min(loss)

In [444]:
o, l=NNClassifier(X_train, t_train,hidden_layer_sizes=(3,5), initilize = random_integers, epochs=10,learning_rate=0.005)
print(l)


1.4869775472190112


In [445]:
print(o)

[7.13467632e-02 7.31058579e-01 1.84745913e-01 7.31058579e-01
 7.31058579e-01 7.11990413e-02 7.31058579e-01 5.23813768e-02
 7.31058579e-01 6.38081881e-01 7.31058579e-01 1.69894905e-03
 5.30421507e-04 4.69825977e-02 5.35418277e-03 7.31058579e-01
 1.42936163e-01 4.49897579e-02 7.31058579e-01 6.22202996e-01
 1.53658617e-02 1.37387939e-01 2.29686896e-02 1.14902553e-01
 7.31058579e-01 5.46238350e-01 1.75321229e-04 6.16091879e-06
 2.15064591e-04 3.75567812e-01 7.31058579e-01 5.23345316e-04
 6.85180900e-01 9.89117307e-02 2.46863535e-06 7.31058579e-01
 5.72018321e-05 7.31058579e-01 7.31058579e-01 9.23866446e-04
 6.74948118e-01 7.69946689e-04 2.79970524e-01 1.84026677e-02
 1.52501781e-01 3.68657527e-05 7.31058579e-01 2.96270767e-02
 7.50872404e-02 5.57413690e-01 3.08085462e-05 1.51823058e-04
 8.50321392e-02 7.31058579e-01 1.33997575e-01 7.31058579e-01
 7.31058579e-01 9.84148231e-03 7.31058579e-01 7.31058579e-01
 5.03036450e-01 7.31058579e-01 4.57790708e-05 5.30422496e-04
 5.87502939e-01 7.113271